In [1]:
VAL = 0.20

In [2]:
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd()

import json
import shutil
import random

# Defining the datasets directory
dataset_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

# Create validation directory if it doesn't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)

# Loading the classes of each image into the memory
train_classes_json_file_name = 'train_gt.json'
train_classes_json_directory = os.path.join(dataset_dir, train_classes_json_file_name)

data = {}

with open(train_classes_json_directory) as json_file:
    data = json.load(json_file)


# Creating folder for each class of image for training and validation datasets
classes = set(data.values())
print(classes)

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    if not os.path.exists(class_training_dir):
        os.makedirs(class_training_dir)
    if not os.path.exists(class_validation_dir):
        os.makedirs(class_validation_dir)

# Assigning images to each training folder/class, avoiding to have the same image two times in the same folder
for entry in os.scandir(training_dir):
    if(entry.is_file()):
        file_destination = os.path.join(training_dir, str(data[entry.name]), entry.name)
        if not os.path.isfile(file_destination):
            shutil.copy(entry.path, file_destination)
    
# Choosing random images to be into the validation folders, being able to repeat without cloning images
validation_rate = VAL

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    
    for old_entry in os.scandir(class_validation_dir):
        os.remove(old_entry.path)
    
    training_entries = list(os.scandir(class_training_dir))
    validation_size = round(len(training_entries)*validation_rate)
    
    for validation_entry in random.sample(training_entries, validation_size):
        destination = os.path.join(class_validation_dir, validation_entry.name)
        os.rename(validation_entry.path, destination)

{0, 1, 2}


In [3]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=0,
        rescale=1/255.
    )
else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
# test_data_gen = ImageDataGenerator(rescale=1/255.)

bs = 8

train_gen = train_data_gen.flow_from_directory(
    training_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_gen = valid_data_gen.flow_from_directory(
    validation_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# test_gen = test_data_gen.flow_from_directory(
#     test_dir,
#     batch_size=bs,
#     class_mode='categorical',
#     shuffle=True,
#     seed=SEED
# )

img_h = 256
img_w = 256

num_classes = len(classes)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

valid_dataset = valid_dataset.repeat()

# test_dataset = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types=(tf.float32, tf.float32),
#     output_shapes=([None, 256, 256, 3], [None, num_classes])
# )

# test_dataset = test_dataset.repeat()

Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


# Building the Network

In [4]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [5]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [6]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               16777728  
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 31,493,955
Trainable params: 23,858,691
Non-trainable params: 7,635,264
_________________________________________________________________


0e-04, -3.09621776e-03, -3.01808352e-03],
          [-2.62522907e-03,  3.51887429e-03, -2.58309790e-03, ...,
           -6.02027494e-03, -8.85035843e-03,  9.85825085e-04]],
 
         [[ 3.02846078e-04,  2.64736195e-03, -1.37099335e-02, ...,
            1.49135189e-02, -4.65666235e-04, -6.84898719e-03],
          [-9.60117579e-03,  5.01031429e-03, -1.13053191e-02, ...,
            8.86707660e-03,  6.21224102e-03,  2.48882570e-03],
          [-1.13499342e-02, -4.72285505e-03, -8.57903156e-03, ...,
           -3.04920389e-03,  9.67197306e-03,  1.90250184e-02],
          ...,
          [-4.48127137e-03,  5.53716440e-03,  4.68912302e-03, ...,
           -4.91005788e-03,  6.40070112e-03, -8.28019064e-03],
          [-6.00830279e-03, -7.51605199e-04,  8.29616503e-04, ...,
            2.07461347e-03,  3.22994636e-03, -1.69727230e-03],
          [-2.81035155e-03,  1.36994272e-02, -2.04460253e-03, ...,
           -5.17683988e-03, -9.04289633e-03, -1.79657899e-03]],
 
         [[ 3.09069827e-03,

In [7]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with callbacks


In [8]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = '/content/drive/My Drive/Keras3'

exps_dir = os.path.join(cwd, 'transfer_learning_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


In [9]:
model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Train for 562 steps, validate for 141 steps
Epoch 1/30
562/562 [==============================] - 344s 612ms/step - loss: 0.9268 - accuracy: 0.5252 - val_loss: 0.7592 - val_accuracy: 0.6194
Epoch 2/30
562/562 [==============================] - 337s 600ms/step - loss: 0.7117 - accuracy: 0.6618 - val_loss: 0.6235 - val_accuracy: 0.6845
Epoch 3/30
562/562 [==============================] - 337s 600ms/step - loss: 0.6525 - accuracy: 0.6950 - val_loss: 0.6237 - val_accuracy: 0.6952
Epoch 4/30
562/562 [==============================] - 336s 598ms/step - loss: 0.6056 - accuracy: 0.7193 - val_loss: 0.5982 - val_accuracy: 0.7157
Epoch 5/30
562/562 [==============================] - 337s 600ms/step - loss: 0.5600 - accuracy: 0.7378 - val_loss: 0.5879 - val_accuracy: 0.7531
Epoch 6/30
562/562 [==============================] - 340s 604ms/step - loss: 0.5339 - accuracy: 0.7533 - val_loss: 0.4974 - val_accuracy: 0.7594
Epoch 7/30
562/562 [==============================] - 336s 597ms/step - loss: 0.

In [10]:
import pandas as pd
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results,dataset_dir)

Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
57/57 [==============================] - 23s 397ms/step


In [11]:
model.fit(x=train_dataset,
          epochs=15,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Train for 562 steps, validate for 141 steps
Epoch 1/15
562/562 [==============================] - 338s 601ms/step - loss: 0.1724 - accuracy: 0.9328 - val_loss: 0.6044 - val_accuracy: 0.8200
Epoch 2/15
562/562 [==============================] - 339s 603ms/step - loss: 0.1623 - accuracy: 0.9352 - val_loss: 0.5681 - val_accuracy: 0.8137
Epoch 3/15
562/562 [==============================] - 361s 642ms/step - loss: 0.1530 - accuracy: 0.9426 - val_loss: 0.5235 - val_accuracy: 0.8333
Epoch 4/15
562/562 [==============================] - 337s 600ms/step - loss: 0.1520 - accuracy: 0.9459 - val_loss: 0.5073 - val_accuracy: 0.8494
Epoch 5/15
562/562 [==============================] - 337s 600ms/step - loss: 0.1679 - accuracy: 0.9386 - val_loss: 0.5371 - val_accuracy: 0.8414
Epoch 6/15
562/562 [==============================] - 337s 601ms/step - loss: 0.1471 - accuracy: 0.9441 - val_loss: 0.4575 - val_accuracy: 0.8512
Epoch 7/15
562/562 [==============================] - 338s 601ms/step - loss: 0.

In [12]:
import pandas as pd
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results,dataset_dir)

Found 450 validated image filenames belonging to 1 classes.
57/57 [==============================] - 24s 416ms/step


## Desnet model


In [13]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=0,
        rescale=1/255.
    )
else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
# test_data_gen = ImageDataGenerator(rescale=1/255.)

bs = 8

train_gen = train_data_gen.flow_from_directory(
    training_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_gen = valid_data_gen.flow_from_directory(
    validation_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# test_gen = test_data_gen.flow_from_directory(
#     test_dir,
#     batch_size=bs,
#     class_mode='categorical',
#     shuffle=True,
#     seed=SEED
# )

img_h = 256
img_w = 256

num_classes = len(classes)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, num_classes])
)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, num_classes])
)

valid_dataset = valid_dataset.repeat()

# test_dataset = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types=(tf.float32, tf.float32),
#     output_shapes=([None, 256, 256, 3], [None, num_classes])
# )

# test_dataset = test_dataset.repeat()

Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


In [4]:
from keras.datasets import cifar10
from keras.applications import DenseNet121, ResNet152, ResNet152V2
from keras.applications import imagenet_utils
from tensorflow.keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from keras.engine import Model
from sklearn.metrics import classification_report
import numpy as np
from keras.layers import Dropout
input_shape = (256, 256, 3)
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers: 
  layer.trainable = False
  print('Capa ' + layer.name + ' freeze...')
  last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(200, activation='relu', name='fc2')(x)
x = Dense(3, activation='softmax', name='predictions')(x)
model_resnet = Model(base_model.input, x)

Using TensorFlow backend.
234700800/234698864 [==============================] - 175s 1us/step
Capa input_1 freeze...
Capa conv1_pad freeze...
Capa conv1_conv freeze...
Capa conv1_bn freeze...
Capa conv1_relu freeze...
Capa pool1_pad freeze...
Capa pool1_pool freeze...
Capa conv2_block1_1_conv freeze...
Capa conv2_block1_1_bn freeze...
Capa conv2_block1_1_relu freeze...
Capa conv2_block1_2_conv freeze...
Capa conv2_block1_2_bn freeze...
Capa conv2_block1_2_relu freeze...
Capa conv2_block1_0_conv freeze...
Capa conv2_block1_3_conv freeze...
Capa conv2_block1_0_bn freeze...
Capa conv2_block1_3_bn freeze...
Capa conv2_block1_add freeze...
Capa conv2_block1_out freeze...
Capa conv2_block2_1_conv freeze...
Capa conv2_block2_1_bn freeze...
Capa conv2_block2_1_relu freeze...
Capa conv2_block2_2_conv freeze...
Capa conv2_block2_2_bn freeze...
Capa conv2_block2_2_relu freeze...
Capa conv2_block2_3_conv freeze...
Capa conv2_block2_3_bn freeze...
Capa conv2_block2_add freeze...
Capa conv2_block2_

ResourceExhaustedError: OOM when allocating tensor with shape[131072,1000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: fc1/random_uniform/

In [15]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in desnet.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
modeldes = tf.keras.Sequential()
modeldes.add(desnet)
modeldes.add(tf.keras.layers.Flatten())
modeldes.add(tf.keras.layers.Dense(units=512, activation='relu'))
modeldes.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
modeldes.summary()

# Visualize initialized weights
modeldes.weights

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Model)            (None, 1000)              60419944  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               512512    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 1539      
Total params: 60,933,995
Trainable params: 60,610,091
Non-trainable params: 323,904
_________________________________________________________________


9556797e-02,
        -3.03886831e-03,  5.59574319e-03, -6.19558292e-03, -1.44089423e-02,
        -1.56995319e-02, -1.94675308e-02,  1.35611498e-03,  4.09621513e-03,
        -5.61026763e-03,  1.76017580e-03, -1.12154484e-02,  7.14821974e-03,
        -2.15429394e-03, -4.72499349e-04, -7.70794926e-03, -9.33720917e-03,
        -5.24849305e-03, -6.99713535e-04,  1.87406298e-02,  3.86947626e-03,
        -7.53889792e-03, -1.10653955e-02,  9.53404885e-03, -3.97438556e-03,
        -1.01516489e-03, -9.80561972e-03, -4.98154899e-03,  1.15464767e-02,
        -2.77407817e-03,  4.13622148e-03, -2.49644020e-03, -6.09258981e-03,
        -5.14376117e-03, -1.90166419e-03,  1.55179966e-02, -1.01111201e-03,
        -2.02945853e-03, -3.01633426e-03,  7.23772589e-03, -6.00466831e-03,
         1.67525057e-02,  1.02070207e-02,  1.69195724e-03, -3.95877054e-04,
         3.42533225e-03,  7.03724008e-03, -2.65088701e-03,  1.61548350e-02,
         1.24175847e-02,  1.32450170e-03, -5.92421321e-03,  3.26480605e-02,

In [16]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
modeldes.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [17]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = '/content/drive/My Drive/Keras3'

exps_dir = os.path.join(cwd, 'transfer_learning_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


In [18]:
modeldes.fit(x=train_gen,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

  ...
    to  
  ['...']
Train for 562 steps, validate for 141 steps
Epoch 1/30
  1/562 [..............................] - ETA: 9:14:30

ResourceExhaustedError:  OOM when allocating tensor with shape[512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/resnet152/conv3_block3_3_bn/FusedBatchNormV3 (defined at <ipython-input-18-5b3f610e54e1>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_282415]

Function call stack:
distributed_function


In [28]:
type(train_dataset)


tensorflow.python.data.ops.dataset_ops.RepeatDataset

In [29]:
type(train_gen)

keras_preprocessing.image.directory_iterator.DirectoryIterator